# Address Assignment and Pathfinding Evaluation

This notebook performs two main tasks:
1. **Address Assignment**: Uses H3 indexing to assign addresses to buildings based on their proximity to landmarks.
2. **Pathfinding Evaluation**: Compares the performance of Dijkstra's, A*, and Bellman-Ford algorithms for finding paths in a road network.

The notebook is structured as follows:
- **Geospatial Data Processing**: Includes functions for H3 indexing and address assignment.
- **Pathfinding Algorithms**: Compares the performance of pathfinding algorithms.


## Geospatial Data Processing
This section includes functions for handling geospatial data using H3 indexing. It involves precomputing H3 indices and assigning addresses to buildings based on proximity to landmarks.

In [19]:
import h3
import pandas as pd
from multiprocessing import Pool, cpu_count
from tqdm import tqdm


def precompute_h3_indices(data, lat_col, lon_col, hex_resolution):
    """
    Precompute H3 indices for a dataset.
    
    Args:
        data (pd.DataFrame): DataFrame containing latitude and longitude columns.
        lat_col (str): Name of the latitude column.
        lon_col (str): Name of the longitude column.
        hex_resolution (int): H3 resolution for indexing.
    
    Returns:
        pd.DataFrame: Updated DataFrame with H3 indices.
    """
    data['h3_index'] = data.apply(lambda row: h3.geo_to_h3(row[lat_col], row[lon_col], hex_resolution), axis=1)
    return data


def assign_address_to_building(args):
    """
    Assign an address to a single building using precomputed H3 indices.
    
    Args:
        args (tuple): Contains building row, landmarks DataFrame, and hex resolution.
    
    Returns:
        dict: Contains the building coordinates, assigned address, and grid reference.
    """
    building, landmarks_df = args
    try:
        building_h3 = building['h3_index']
        matches = landmarks_df[landmarks_df['h3_index'] == building_h3]

        if not matches.empty:
            closest_landmark = matches.iloc[0]
            landmark_name = closest_landmark['tags_name']
            address = f"Near {landmark_name}, Grid {building_h3}"
        else:
            address = "Address not assigned"

        return {
            'latitude': building['latitude'],
            'longitude': building['longitude'],
            'address': address
        }
    except Exception as e:
        return {
            'latitude': building['latitude'],
            'longitude': building['longitude'],
            'address': f"Error: {str(e)}"
        }


def assign_address_units_optimized(buildings_df, landmarks_df, hex_resolution=7):
    """
    Optimized function to assign addresses to buildings using multiprocessing and precomputed H3 indices.
    
    Args:
        buildings_df (pd.DataFrame): DataFrame containing building data with latitude and longitude.
        landmarks_df (pd.DataFrame): DataFrame containing landmark data with latitude, longitude, and names.
        hex_resolution (int): Resolution for H3 hex indexing.
    
    Returns:
        pd.DataFrame: DataFrame with assigned address units.
    """
    # Precompute H3 indices
    buildings_df = precompute_h3_indices(buildings_df, 'latitude', 'longitude', hex_resolution)
    landmarks_df = precompute_h3_indices(landmarks_df, 'lat', 'lon', hex_resolution)

    # Prepare arguments for multiprocessing
    args = [(building, landmarks_df) for _, building in buildings_df.iterrows()]

    # Use multiprocessing to parallelize the address assignment
    with Pool(cpu_count()) as pool:
        results = list(tqdm(pool.imap(assign_address_to_building, args), total=len(buildings_df), desc="Assigning Addresses"))

    # Convert results back to DataFrame
    result_df = pd.DataFrame(results)
    return result_df


# Sample Usage
buildings_sample = pd.read_csv('./data/kathmandu_buildings.csv')
landmarks_sample = pd.read_csv('./data/cleaned_landmarks.csv')

# Assign addresses using the optimized function
assigned_addresses = assign_address_units_optimized(buildings_sample, landmarks_sample)

# Save the results to a CSV file
assigned_addresses.to_csv('./data/sample_assigned_addresses_optimized.csv', index=False)
print("Optimized address assignment completed. Output saved to 'sample_assigned_addresses_optimized.csv'.")


Assigning Addresses: 100%|██████████| 149054/149054 [05:03<00:00, 491.15it/s]


Optimized address assignment completed. Output saved to 'sample_assigned_addresses_optimized.csv'.


### Results Display
This cell shows the `assigned_addresses` DataFrame containing the results of the address assignment process.

In [20]:
assigned_addresses

,latitude,longitude,address
0,27.714330,85.328456,"Near Tranquility Spa, Grid 873c02991ffffff"
1,27.731412,85.308919,"Near Silo Family Restaurant, Grid 873c02995ffffff"
2,27.687587,85.333969,"Near Beijing Duck, Grid 873c02d6dffffff"
3,27.686682,85.349605,"Near किशोर कुमार सेकुवा, Grid 873c0299affffff"
4,27.720405,85.331616,"Near Tranquility Spa, Grid 873c02991ffffff"
...,...,...,...
149049,27.690034,85.327322,"Near Beijing Duck, Grid 873c02d6dffffff"
149050,27.689717,85.337461,"Near Beijing Duck, Grid 873c02d6dffffff"
149051,27.698889,85.326920,"Near काठमाडौं भान्सा, Grid 873c02993ffffff"
149052,27.702800,85.312127,"Near काठमाडौं भान्सा, Grid 873c02993ffffff"


## Pathfinding Algorithms
This section implements and evaluates three pathfinding algorithms:
1. **Dijkstra's Algorithm**: Finds the shortest path based on edge weights.
2. **A\* Search Algorithm**: An optimized pathfinding algorithm using heuristics.
3. **Bellman-Ford Algorithm**: Handles graphs with negative weights but is slower compared to the others.

In [22]:
import osmnx as ox
import networkx as nx
from tqdm import tqdm
import time

def heuristic_function(node1, node2, graph):
    """
    Wrapper for the heuristic function to calculate the great-circle distance.
    Converts node IDs to coordinates before passing them to great_circle_vec.
    
    Args:
        node1 (int): Node ID of the first point.
        node2 (int): Node ID of the second point.
        graph (networkx.Graph): Graph containing node coordinates.

    Returns:
        float: Great-circle distance between node1 and node2.
    """
    # Retrieve coordinates from the graph
    x1, y1 = graph.nodes[node1]['x'], graph.nodes[node1]['y']
    x2, y2 = graph.nodes[node2]['x'], graph.nodes[node2]['y']
    return ox.distance.great_circle_vec(y1, x1, y2, x2)

def compare_pathfinding_algorithms(graph, start_node, end_node):
    """
    Compare Dijkstra's, A*, and Bellman-Ford algorithms for pathfinding.

    Args:
        graph (networkx.Graph): Road network graph.
        start_node (int): Start node ID.
        end_node (int): End node ID.

    Returns:
        dict: Results for each algorithm including path, travel time, and computational time.
    """
    results = {}

    # Dijkstra's Algorithm
    start_time = time.time()
    dijkstra_path = nx.shortest_path(graph, start_node, end_node, weight='travel_time')
    dijkstra_time = time.time() - start_time
    results['Dijkstra'] = {
        'path': dijkstra_path,
        'time': dijkstra_time
    }

    # A* Algorithm
    start_time = time.time()
    a_star_path = nx.astar_path(
        graph,
        start_node,
        end_node,
        heuristic=lambda u, v: heuristic_function(u, v, graph),
        weight='travel_time'
    )
    a_star_time = time.time() - start_time
    results['A*'] = {
        'path': a_star_path,
        'time': a_star_time
    }

    # Bellman-Ford Algorithm
    try:
        start_time = time.time()
        bellman_path = nx.single_source_bellman_ford_path(graph, start_node, weight='travel_time')[end_node]
        bellman_time = time.time() - start_time
        results['Bellman-Ford'] = {
            'path': bellman_path,
            'time': bellman_time
        }
    except nx.NetworkXUnbounded:
        results['Bellman-Ford'] = {
            'path': None,
            'time': 'Unbounded'
        }

    return results

def evaluate_pathfinding(results):
    """
    Evaluate pathfinding performance based on predefined metrics.

    Args:
        results (dict): Pathfinding results from compare_pathfinding_algorithms.

    Returns:
        pd.DataFrame: DataFrame summarizing the performance metrics.
    """
    metrics = []
    for algo, result in results.items():
        metrics.append({
            'Algorithm': algo,
            'Path Length': len(result['path']) if result['path'] else None,
            'Time (s)': result['time']
        })
    return pd.DataFrame(metrics)

# Example Usage
print("Downloading graph...")
G = ox.graph_from_place('Kathmandu, Nepal', network_type='drive')
nodes = list(G.nodes)
start, end = nodes[0], nodes[-1]  # Replace with specific node IDs if available

print("Comparing pathfinding algorithms...")
path_results = compare_pathfinding_algorithms(G, start, end)

print("Evaluating performance...")
performance_metrics = evaluate_pathfinding(path_results)
performance_metrics.to_csv('./data/pathfinding_metrics.csv', index=False)
print("Performance metrics saved to 'pathfinding_metrics.csv'.")


Comparing pathfinding algorithms...
Evaluating performance...
Performance metrics saved to 'pathfinding_metrics.csv'.


### Final Evaluation
This section summarizes the evaluation results of the pathfinding algorithms, including their path lengths and computational times.

In [23]:
# final evaluation (pathfinding_metrics.csv)

# Algorithm,Path Length,Time (s)
# Dijkstra,61,0.015609264373779297
# A*,84,0.0029883384704589844
# Bellman-Ford,61,0.3031022548675537


## Conclusion
The notebook demonstrates how to use H3 indexing for geospatial data processing and compares the performance of common pathfinding algorithms. Further optimization and visualization can enhance its utility.